In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from researchlib.single_import import *

In [14]:
ctf = [transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip()]

# Load dataset
train_loader = FromPublic('cifar10', 'train', batch_size=512, num_workers=8, custom_tf=ctf)
test_loader = FromPublic('cifar10', 'test', batch_size=512, num_workers=8)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
# Model
model = SENet18(input_dim=3)

# Learning
runner = Runner(model, train_loader, test_loader, optimizer='adam', loss_fn='focal')

In [16]:
for _ in range(3): runner.fit_onecycle(1e-3, mixup_alpha=0.4)

  Epochs    train_loss    train_acc      val_loss      val_acc    
    1        12.1539        0.3712        7.6759        0.5984    


  Epochs    train_loss    train_acc      val_loss      val_acc    
    1         9.7745        0.5117        5.7318        0.6880    


  Epochs    train_loss    train_acc      val_loss      val_acc    
    1         8.5948        0.5830        5.1683        0.7205    


In [11]:
runner.fit_cycle(5, 1e-3, mixup_alpha=0.4)

  Epochs    train_loss    train_acc      val_loss      val_acc    
    1         5.9102        0.7335        2.6587        0.8693    


    2         6.3400        0.7092        2.9568        0.8503    


KeyboardInterrupt: 

In [ ]:
runner.validate(metrics=[ConfusionMatrix(plot=True, classes=10)])

In [ ]:
runner.history(plot=True)